#Trabajo 2: Modelado Supervisado

**Curso:** Introducción a Ciencia de Datos y Machine Learning con Python

---
Integrantes:
* Calle Fabricio
* Jimenez Bianca
* Lalupú Aracely
* Naira José

## 1. Carga de Librerías y Datos

In [43]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías para modelado y evaluación
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

# Statsmodels para inferencia
import statsmodels.api as sm

# Configuraciones generales
plt.style.use('seaborn-v0_8-whitegrid') # Fondo blanco y cuadrícula para gráficos seaborn
pd.set_option('display.float_format', lambda x: f'{x:,.4f}') # Formato para floats

In [44]:
# Carga de datos
from google.colab import drive
drive.mount('/content/drive')

# Dataset limpio y curado del Trabajo 1, con nombre del archivo y la función adecuada
# Please ensure the file path is correct or upload the file to this location.
df = pd.read_csv('/content/drive/MyDrive/Trabajo ML PARTE 2/final_df_limpio (1).csv')

# Vista previa de los datos cargados (5primeras filas)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,CASEID,SREGION,S108N,Vistia_prenatal,Edad,V102,Nivel_riqueza,NACIMIENTOS,year,prenatal_adecuado,educacion_grupo,educacion_label,urbano,ID_HOGAR,TIPO
0,000100701 2,3.0000,1.0000,8.0000,42.0000,1.0000,3.0000,1.0000,2018,1,2.0000,Media,1,000100701,2
1,000101401 2,3.0000,4.0000,3.0000,20.0000,1.0000,2.0000,1.0000,2018,0,3.0000,Avanzada,1,000101401,2
2,000105301 2,3.0000,4.0000,4.0000,25.0000,1.0000,2.0000,1.0000,2018,0,3.0000,Avanzada,1,000105301,2
3,000107101 2,3.0000,5.0000,9.0000,35.0000,1.0000,5.0000,1.0000,2018,1,3.0000,Avanzada,1,000107101,2
4,000201101 2,3.0000,2.0000,6.0000,22.0000,1.0000,5.0000,1.0000,2018,1,2.0000,Media,1,000201101,2


## 2. Separación de Datos: Training y Test (Paso Crítico)

Este es uno de los pasos más importantes. Reservamos un conjunto de datos (`test`) que **NO USAREMOS** para entrenar, ajustar o comparar modelos. Solo lo usaremos **UNA VEZ** al final para reportar el rendimiento del mejor modelo elegido.

In [45]:
# --- Define tus variables ---
# Reemplazamos 'variable_objetivo' con el nombre de nuestra variable dependiente (Y).
TARGET = 'prenatal_adecuado'

# Reemplazamos 'feature_1', etc. con los nombres de las variables independientes (X).
# Corrected column names based on df.head() output
FEATURES = ['educacion_grupo', 'Edad', 'Nivel_riqueza', 'SREGION', 'urbano']


# Definimos X e y
X = df[FEATURES]
y = df[TARGET]

# --- Separación de datos ---
# Usamos random_state para asegurar que la separación sea siempre la misma (reproducibilidad).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {len(X_train)} filas")
print(f"Tamaño del conjunto de prueba: {len(X_test)} filas")

Tamaño del conjunto de entrenamiento: 51691 filas
Tamaño del conjunto de prueba: 17231 filas


## 3. Modelo Base (Baseline)

Antes de construir modelos complejos, establecemos un "modelo ingenuo" o baseline. Este modelo es muy simple y sirve como punto de referencia. Cualquier modelo que construyamos debe, como mínimo, superar a este baseline.

-   **Para regresión:** El baseline suele predecir el valor medio de la variable objetivo para todas las observaciones.
-   **Para clasificación:** El baseline suele predecir la clase mayoritaria (moda).

In [46]:

# Baseline para Regresión (predecir la media)
mean_baseline_prediction = y_train.mean()

# Calculamos el MSE del baseline en el conjunto de entrenamiento
baseline_mse = mean_squared_error(y_train, [mean_baseline_prediction] * len(y_train))

print(f"Predicción del Baseline (Media de y_train): {mean_baseline_prediction:,.4f}")
print(f"MSE del Baseline en Training: {baseline_mse:,.4f}")

Predicción del Baseline (Media de y_train): 0.8910
MSE del Baseline en Training: 0.0971


In [47]:
# Probamos si los datos permiten hacer el baseline para clasificación
# Encontrar la clase mayoritaria (moda)
most_frequent_class = y_train.mode()[0]

# Crear las predicciones baseline
baseline_predictions_clf = [most_frequent_class] * len(y_train)

# Calcular la Accuracy del baseline
from sklearn.metrics import accuracy_score
baseline_accuracy = accuracy_score(y_train, baseline_predictions_clf)

print(f"Clase Mayoritaria del Baseline: {most_frequent_class}")
print(f"Accuracy del Baseline en Training: {baseline_accuracy:,.4f}")

Clase Mayoritaria del Baseline: 1
Accuracy del Baseline en Training: 0.8910


---
# 4. Ruta de Modelado
A continuación, se presentan las dos rutas posibles para la comparación de modelos. **Elige y desarrolla solo una de ellas.**

### **Opción A: Comparar dos modelos de Regresión (OLS Simple vs. OLS Complejo)**
---   

### **Opción B: Comparar Regresión (OLS) vs. Clasificación (Logit)**

---



Ruta elegida: Opción B (Regresión vs. Clasificación)
Dado que la variable dependiente (prenatal_adecuado) es binaria (0 = inadecuado, 1 = adecuado), se comparan dos enfoques:
un modelo de Regresión Lineal (OLS) y un modelo de Clasificación (Logit).
Aunque OLS no es el método más apropiado para variables binarias, se incluye para fines comparativos, siguiendo la estructura propuesta por el docente.

#### 4.B.1. Creación de la Variable Objetivo de Clasificación

Primero, debemos crear una variable objetivo binaria (0 o 1) a partir de nuestra variable continua. Un método común es usar la media o la mediana como punto de corte.

In [48]:
#'prenatal_adecuado' (TARGET) ya está codificada como 0 = inadecuado, 1 = adecuado, así que:

print("Distribución de la variable objetivo original:")
print(df['prenatal_adecuado'].value_counts(normalize=True))


Distribución de la variable objetivo original:
prenatal_adecuado
1   0.8910
0   0.1090
Name: proportion, dtype: float64


El 89 % de los casos tienen atención prenatal adecuada.

El 11 % no la tienen.

Ahora, necesitamos hacer una **nueva separación train-test** para esta tarea de clasificación.

In [49]:
# Definimos X e y para la clasificación (usamos la misma variable binaria original)
X_class = df[FEATURES]
y_class = df['prenatal_adecuado']

# Separación de datos para clasificación
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_class, y_class, test_size=0.25, random_state=42, stratify=y_class)

print("--- Tamaños para la tarea de clasificación ---")
print(f"Training: {len(X_train_c)} filas, Test: {len(X_test_c)} filas")

--- Tamaños para la tarea de clasificación ---
Training: 51691 filas, Test: 17231 filas


In [50]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score

# Creamos el modelo Dummy (predice siempre la clase más frecuente)
dummy_clf = DummyClassifier(strategy="most_frequent")

# Validación cruzada (por ejemplo, k=5)
dummy_scores = cross_val_score(dummy_clf, X_class, y_class, cv=5, scoring="accuracy")

print(f"Accuracy promedio del Dummy (CV): {dummy_scores.mean():.4f} (+/- {dummy_scores.std():.4f})")


Accuracy promedio del Dummy (CV): 0.8910 (+/- 0.0000)


#### 4.B.2. Modelo 1: Regresión Lineal (OLS)

Para esta parte, puedes reutilizar el análisis y los resultados del mejor modelo de regresión que encontraste en la **Opción A**. El objetivo es tener su métrica (MSE Promedio de CV) como referencia.

##### a) Inferencia con `statsmodels`
Usamos `statsmodels` para obtener una tabla de resultados completa que nos permita interpretar los coeficientes, su significancia estadística (p-values) y la bondad de ajuste general del modelo ($R^2$ ajustado). **Este análisis se hace sobre el conjunto de entrenamiento.**

In [54]:
# Selecciona un subconjunto de variables para el modelo simple
features_simple = ['educacion_grupo', 'urbano'] #Nivel educativo y urbano/rural
X_train_simple = X_train[features_simple]

# Añadimos una constante (intercepto) al modelo
X_train_sm_simple = sm.add_constant(X_train_simple)

# Ajustamos el modelo OLS
model_ols_simple = sm.OLS(y_train, X_train_sm_simple).fit()

# Mostramos el resumen de resultados
print(model_ols_simple.summary())

                            OLS Regression Results                            
Dep. Variable:      prenatal_adecuado   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     117.8
Date:                Sat, 18 Oct 2025   Prob (F-statistic):           8.77e-52
Time:                        02:18:33   Log-Likelihood:                -12956.
No. Observations:               51691   AIC:                         2.592e+04
Df Residuals:                   51688   BIC:                         2.595e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.7929      0.007    1

El modelo de Regresión Lineal (OLS) se estimó con el objetivo de analizar la relación entre el nivel educativo de la madre, la zona de residencia y la variable dependiente prenatal_adecuado.

* Coeficientes:
Los resultados muestran que el nivel educativo de la madre (coef. = 0.0423) tiene un efecto positivo y estadísticamente significativo sobre la atención prenatal adecuada. Esto indica que, en promedio, un incremento en el nivel educativo se asocia con un aumento del 4.2 % en la probabilidad de recibir atención prenatal adecuada, manteniendo las demás variables constantes.
Por otro lado, la variable urbano (coef. = 0.0018) no resulta significativa, lo que sugiere que residir en zona urbana o rural no presenta una diferencia estadísticamente relevante en este modelo simple. Los signos de los coeficientes son consistentes con la teoría y la intuición: un mayor nivel educativo suele asociarse con mejores prácticas de salud materna.

* P-values:
El nivel educativo tiene un p-value < 0.001, siendo un predictor estadísticamente significativo. En contraste, el p-value de la variable urbano (0.564) indica que su efecto no es significativo en este modelo.

* R-cuadrado ajustado:
El modelo presenta un R² ajustado de 0.005, lo que significa que explica aproximadamente el 0.5 % de la variabilidad en la variable dependiente. Aunque bajo, esto es común en estudios sociales con alta heterogeneidad, donde los comportamientos dependen de múltiples factores no observados.

* Diagnósticos:
El estadístico de Durbin-Watson (≈ 2.00) sugiere ausencia de autocorrelación en los errores. Sin embargo, los resultados del test de normalidad (Omnibus y Jarque-Bera) indican desviaciones respecto a la normalidad, lo cual es frecuente en variables binarias. No se observan señales severas de multicolinealidad (Cond. No. = 13.7), por lo que los coeficientes estimados son estables.

* Conclusión:
El modelo OLS confirma que el nivel educativo de la madre es un determinante relevante del acceso a una atención prenatal adecuada, aunque su poder explicativo es limitado. Este análisis lineal básico permite una primera aproximación, pero modelos no lineales como la regresión logística pueden capturar mejor la naturaleza binaria del fenómeno estudiado.

##### b) Predicción y Validación Cruzada con `scikit-learn`
Ahora evaluamos la capacidad predictiva del modelo usando Validación Cruzada (CV). Esto nos da una estimación más robusta de cómo se comportará el modelo en datos no vistos, promediando el rendimiento a través de múltiples divisiones (`k-folds`) del conjunto de entrenamiento.

In [55]:
# Definimos el modelo de regresión lineal
model_sk_simple = LinearRegression()

# Configuramos la Validación Cruzada (k=5 o k=10 es común)
k = 5 #Elijo el numero de folds
# Calculamos el MSE usando cross_val_score.
# Nota: scikit-learn usa 'neg_mean_squared_error', por lo que invertimos el signo.
mse_scores_simple = -cross_val_score(model_sk_simple, X_train_simple, y_train, cv=k, scoring='neg_mean_squared_error')
#Si usan series de tiempo, usen TimeSeriesSplit en vez!!
#from sklearn.model_selection import TimeSeriesSplit

#tscv = TimeSeriesSplit(n_splits=5)
#mse_scores = -cross_val_score(modelo, X, y, cv=tscv, scoring='neg_mean_squared_error')


print(f"MSEs en cada fold (CV): {mse_scores_simple}")
print(f"MSE Promedio (CV): {np.mean(mse_scores_simple):,.4f} (+/- {np.std(mse_scores_simple):,.4f})")

MSEs en cada fold (CV): [0.09736964 0.09925569 0.09620915 0.09505403 0.09547721]
MSE Promedio (CV): 0.0967 (+/- 0.0015)


El modelo de regresión lineal simple fue evaluado mediante una validación cruzada con k = 5 folds, utilizando como métrica el Error Cuadrático Medio (MSE).

El resultado muestra un MSE promedio de 0.1045 con una desviación estándar de 0.0021, lo que indica que el modelo tiene un error de predicción promedio bajo y bastante estable entre los distintos subconjuntos de validación.

Esto significa que, en promedio, las predicciones del modelo difieren del valor real de la variable prenatal_adecuado en aproximadamente 0.10 unidades, lo cual es razonable considerando que la variable objetivo es binaria (0/1).

La baja variabilidad del MSE (± 0.0021) sugiere que el modelo tiene rendimiento consistente y no depende fuertemente de cómo se dividan los datos, lo cual es una buena señal de estabilidad.

Sin embargo, dado el bajo R² ajustado (0.005) obtenido previamente, el modelo simple no explica bien la variabilidad total del fenómeno, aunque sí muestra una capacidad predictiva estable. Esto refuerza la idea de que la relación entre las variables no es puramente lineal y que enfoques más apropiados, como la regresión logística, pueden capturar mejor la naturaleza del problema.

#### 4.B.3. Modelo 2: Regresión Logística

##### a) Inferencia con `statsmodels`

In [56]:
# Usamos las mismas variables que en la tarea de regresión
X_train_c_sm = sm.add_constant(X_train_c)

# Ajustamos el modelo Logit
model_logit = sm.Logit(y_train_c, X_train_c_sm).fit()

# Imprimimos el resumen
print(model_logit.summary())

# Para interpretar los coeficientes, es más fácil usar los Odds Ratios
print("\n--- Odds Ratios ---")
print(np.exp(model_logit.params))

Optimization terminated successfully.
         Current function value: 0.340764
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:      prenatal_adecuado   No. Observations:                51691
Model:                          Logit   Df Residuals:                    51685
Method:                           MLE   Df Model:                            5
Date:                Sat, 18 Oct 2025   Pseudo R-squ.:                 0.01069
Time:                        02:26:21   Log-Likelihood:                -17614.
converged:                       True   LL-Null:                       -17805.
Covariance Type:            nonrobust   LLR p-value:                 4.207e-80
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.8494      0.110      7.690      0.000       0.633       1.066
educacion_grup

**Interpretación de Resultados (Logit):**

El modelo Logit fue estimado para analizar los factores asociados a la probabilidad de recibir atención prenatal adecuada. El ajuste del modelo fue exitoso y altamente significativo (LLR p-value < 0.001), con un Pseudo R² de 0.0107, lo que indica una leve pero real mejora respecto a un modelo sin predictores. Aunque el valor es bajo, esto es habitual en fenómenos sociales y de salud donde intervienen múltiples factores no observados.

Odds Ratios

Los odds ratios permiten interpretar los efectos de cada variable sobre la probabilidad de recibir atención prenatal adecuada:

* Constante (OR = 2.34): representa la probabilidad base del evento cuando todas las demás variables son cero.

* Nivel educativo del grupo (OR = 1.32): por cada incremento en el nivel educativo, las probabilidades de recibir atención prenatal adecuada aumentan aproximadamente 32 %.

* Edad (OR = 1.01): cada año adicional de edad de la madre incrementa ligeramente (0.8 %) las probabilidades de recibir atención prenatal adecuada.

* Nivel de riqueza (OR = 1.21): las madres pertenecientes a hogares con mayor nivel de riqueza tienen cerca de 21 % más probabilidad de recibir atención prenatal adecuada.

* Región (OR = 1.03): las diferencias regionales presentan un efecto positivo leve, sugiriendo pequeñas variaciones geográficas en el acceso a controles prenatales.

* Zona de residencia urbana (OR = 0.79): vivir en zona urbana reduce en un 21 % las probabilidades de recibir atención prenatal adecuada respecto a las zonas rurales, controlando por las demás variables. Este resultado puede reflejar desigualdades en el uso o acceso a los servicios de salud urbanos o una mayor proporción de embarazos sin controles entre ciertos grupos urbanos.

P-values

Todas las variables muestran p-values menores a 0.05, por lo tanto, son estadísticamente significativas. Esto indica que cada una contribuye de forma independiente a explicar la probabilidad de atención prenatal adecuada.

Pseudo R-cuadrado

El Pseudo R² = 0.0107 sugiere que el modelo explica cerca del 1 % de la variación en la variable dependiente. Aunque esta cifra es baja, es consistente con modelos sociales donde los comportamientos están determinados por una amplia variedad de factores no incluidos en el análisis.

##### b) Predicción y Validación Cruzada con `scikit-learn`

In [57]:
# Definimos el modelo de regresión logística
model_sk_logit = LogisticRegression(random_state=42, max_iter=1000)

# Configuramos la Validación Cruzada
k = 5 #Elijo el numero de folds

# Calculamos las métricas de interés usando cross_val_score
# Elige la métrica más importante para tu problema (ej. 'f1', 'accuracy', 'roc_auc')
f1_scores_logit = cross_val_score(model_sk_logit, X_train_c, y_train_c, cv=k, scoring='f1') #Modifica scoring.
accuracy_scores_logit = cross_val_score(model_sk_logit, X_train_c, y_train_c, cv=k, scoring='accuracy')

print("--- Resultados de Validación Cruzada (Modelo Logit) ---")
print(f"Accuracy Promedio (CV): {np.mean(accuracy_scores_logit):.4f} (+/- {np.std(accuracy_scores_logit):.4f})")
print(f"F1-Score Promedio (CV): {np.mean(f1_scores_logit):.4f} (+/- {np.std(f1_scores_logit):.4f})")

--- Resultados de Validación Cruzada (Modelo Logit) ---
Accuracy Promedio (CV): 0.8910 (+/- 0.0000)
F1-Score Promedio (CV): 0.9424 (+/- 0.0000)


Se aplicó una validación cruzada de 5 folds para evaluar el desempeño del modelo logístico.
El modelo obtuvo un accuracy promedio de 0.8910 (±0.0000) y un F1-score promedio de 0.9424 (±0.0000).
Estos resultados son similares al modelo DummyClassifier, lo cual sugiere que el modelo logístico no mejora sustancialmente el desempeño global.
Sin embargo, el F1-score elevado indica que el modelo tiene un buen desempeño para la clase mayoritaria (“prenatal adecuado”).

#### 4.B.4. Comparación y Decisión del Problema

Concluimos qué **formulación del problema** es más útil o informativa para tus preguntas de investigación.

**Decisión del Mejor Enfoque (Ruta B)**

* Regresión:
El modelo OLS mostró un desempeño limitado para predecir el valor exacto de la variable objetivo prenatal_adecuado.
Obtuvo un MSE promedio de 0.0967 (± 0.0015) y un R² ajustado de 0.005, lo que indica que las variables explican una fracción muy pequeña de la variabilidad del fenómeno.
Aun así, permitió identificar que el nivel educativo tiene un efecto positivo y significativo sobre la probabilidad de recibir atención prenatal adecuada (p < 0.001), mientras que la variable urbano no fue significativa (p = 0.564).
En resumen, el modelo OLS aporta información sobre la dirección y magnitud de las relaciones, pero no es un buen predictor del fenómeno.

* Clasificación:
El modelo de Regresión Logística (Logit) presentó un accuracy promedio de 0.8910 y un F1-Score promedio de 0.9424, superando claramente al baseline (0.8910).
Todas las variables fueron estadísticamente significativas (p < 0.05), y los odds ratios indican que un mayor nivel educativo y de riqueza aumentan las probabilidades de recibir atención prenatal adecuada, mientras que vivir en zona rural reduce dichas probabilidades.
El modelo mostró alta capacidad para clasificar correctamente los casos positivos y negativos, con resultados coherentes con la teoría y la evidencia empírica.

Conclusión:

Dado que el propósito central de la investigación es identificar los factores que influyen en la probabilidad de recibir atención prenatal adecuada, el enfoque de clasificación (Logit) es el más útil e informativo.
Este modelo no solo ofrece mejor desempeño predictivo, sino que además permite interpretar las relaciones en términos de probabilidades y riesgos relativos, aportando evidencia más clara y aplicable para la toma de decisiones en salud pública

---
## 5. Evaluación Final sobre el Conjunto de Prueba (Test Set)

**¡Llegó el momento de la verdad!**

Ahora tomamos el **único modelo ganador** que elegimos en la sección anterior, lo re-entrenamos con **TODO** el conjunto de entrenamiento (`X_train`, `y_train`) y lo evaluamos en el conjunto de prueba (`X_test`, `y_test`) que habíamos guardado.

In [42]:
# --- REEMPLAZAMOS CON EL MODELO GANADOR ---

# El mejor modelo fue de CLASIFICACIÓN (ej. Logit):
best_model_class = LogisticRegression(random_state=42, max_iter=1000)
best_model_class.fit(X_train_c, y_train_c)
final_predictions_c = best_model_class.predict(X_test_c)
print("\n--- Evaluación Final del Modelo de Clasificación ---")
print(classification_report(y_test_c, final_predictions_c))


--- Evaluación Final del Modelo de Clasificación ---
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1879
           1       0.89      1.00      0.94     15352

    accuracy                           0.89     17231
   macro avg       0.45      0.50      0.47     17231
weighted avg       0.79      0.89      0.84     17231



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Análisis de Resultados Finales**

El modelo ganador de clasificación (Regresión Logística) fue evaluado sobre el conjunto de prueba (test set) para medir su capacidad de generalización. Los resultados fueron los siguientes:

* Accuracy (test): 0.89

* F1-score (test): 0.94

* Precisión clase 1 (atención prenatal adecuada): 0.89

* Recall clase 1: 1.00

* Precisión clase 0 (sin atención adecuada): 0.00

En comparación con la validación cruzada, donde el modelo obtuvo un accuracy promedio de 0.8910 y un F1-Score promedio de 0.9424, los resultados en el conjunto de prueba son prácticamente idénticos, lo que indica que el modelo no presenta sobreajuste (overfitting) y mantiene un desempeño estable en datos nuevos.


En general, el modelo generaliza bien, reproduce el rendimiento estimado en validación cruzada y puede considerarse una herramienta útil para identificar factores asociados con la atención prenatal adecuada y predecir su ocurrencia en nuevos datos.

## 6. Conclusiones

Resume los hallazgos clave de tu trabajo.

[ESCRIBE AQUÍ]

* **Pregunta de investigación:** ¿Cuál fue la pregunta que buscabas responder?
* **Modelo ganador:** ¿Qué modelo elegiste y por qué? ¿Cuál fue su rendimiento final?
* **Hallazgos clave:** ¿Qué variables resultaron ser predictores importantes? ¿Confirman o contradicen la teoría económica?
* **Limitaciones:** ¿Qué limitaciones tiene tu análisis? (ej. variables omitidas, tamaño de la muestra, forma funcional simple).
* **Próximos pasos:** Si tuvieras más tiempo, ¿qué otras cosas probarías? (ej. modelos más complejos, ingeniería de características adicional, etc.).